In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from mplsoccer.pitch import Pitch
import shapely.geometry as geom

Load the data

In [2]:
WYSCOUT = os.path.join('..', '..', 'data', 'wyscout')
df_wyscout_event = pd.read_parquet(os.path.join(WYSCOUT, 'event.parquet'))
df_wyscout_player = pd.read_parquet(os.path.join(WYSCOUT, 'player.parquet'))
df_wyscout_match = pd.read_parquet(os.path.join(WYSCOUT, 'match.parquet'))

Add on team name

In [3]:
df_team = pd.concat([(df_wyscout_match[['away_team_id', 'away_team_name']]
                      .rename({'away_team_id': 'team_id', 'away_team_name': 'team_name'}, axis=1)),
                     (df_wyscout_match[['home_team_id', 'home_team_name']]
                      .rename({'home_team_id': 'team_id', 'home_team_name': 'team_name'}, axis=1))])
df_team.drop_duplicates('team_id', inplace=True)
df_wyscout_event = df_wyscout_event.merge(df_team, on='team_id', how='left')

Replace player_id = Zero with nullm

In [4]:
df_wyscout_event.player_id.replace({0: np.nan}, inplace=True)

Pitches for coordinate conversion

In [5]:
pitch_wyscout = Pitch(pitch_type='wyscout')
pitch_statsperform = Pitch(pitch_type='statsperform', figsize=(16, 9))

Rename Free Kick to Set Piece

In [6]:
df_wyscout_event.eventName.replace('Free Kick', 'Set Piece', inplace=True)

Add on a column for a pass attempt

In [7]:
mask_pass = ((df_wyscout_event.eventName == 'Pass') | 
             df_wyscout_event.subEventName.isin(['Throw in', 'Free Kick', 'Goal kick', 'Corner', 'Free kick cross']))
df_wyscout_event['pass_attempt'] = mask_pass

Add shot attempt boolean column

In [8]:
mask_corner_goal = (df_wyscout_event.subEventName=='Corner') & (df_wyscout_event.goal==True)
df_wyscout_event['shot'] = ((df_wyscout_event.eventName == 'Shot') | 
                            (df_wyscout_event.subEventName=='Free kick shot') | 
                            (df_wyscout_event.subEventName == 'Penalty') |
                            (mask_corner_goal))

Add on switch (StatsBomb definition = ball transitioned at least 50% of the pitch vertically). Note that I have already removed dodgy end locations near the corner flags so this works.

In [9]:
mask_switch = (abs(df_wyscout_event.end_y - df_wyscout_event.y) >= 50) & (df_wyscout_event.pass_attempt)
df_wyscout_event['pass_switch'] = mask_switch

Add on cross (StatsBomb definition: See Appendix 6 of the Docs for Events)

In [10]:
# cross right side start
cross_right_start = np.array([[pitch_wyscout.right,
                               pitch_wyscout.bottom - pitch_wyscout.six_yard_from_side],
                              [pitch_wyscout.right,
                               pitch_wyscout.bottom],
                              [pitch_wyscout.right - pitch_wyscout.pitch_length*0.3,
                               pitch_wyscout.bottom],
                              [pitch_wyscout.right - pitch_wyscout.pitch_length*0.3,
                               pitch_wyscout.bottom - pitch_wyscout.penalty_area_from_side],
                              [pitch_wyscout.right - pitch_wyscout.penalty_area_length,
                               pitch_wyscout.bottom - pitch_wyscout.penalty_area_from_side]])
cross_right_start = geom.Polygon(cross_right_start)
# cross right side end
cross_right_end = np.array([[pitch_wyscout.right,
                             pitch_wyscout.top],
                            [pitch_wyscout.right - pitch_wyscout.penalty_area_length,
                             pitch_wyscout.top],
                            [pitch_wyscout.right - pitch_wyscout.penalty_area_length,
                             pitch_wyscout.bottom - pitch_wyscout.six_yard_from_side],
                            [pitch_wyscout.right,
                             pitch_wyscout.bottom - pitch_wyscout.six_yard_from_side]])
cross_right_end = geom.Polygon(cross_right_end)
# cross left side start
cross_left_start = np.array([[pitch_wyscout.right,
                               pitch_wyscout.top + pitch_wyscout.six_yard_from_side],
                              [pitch_wyscout.right,
                               pitch_wyscout.top],
                              [pitch_wyscout.right - pitch_wyscout.pitch_length*0.3,
                               pitch_wyscout.top],
                              [pitch_wyscout.right - pitch_wyscout.pitch_length*0.3,
                               pitch_wyscout.top + pitch_wyscout.penalty_area_from_side],
                              [pitch_wyscout.right - pitch_wyscout.penalty_area_length,
                               pitch_wyscout.top + pitch_wyscout.penalty_area_from_side]])
cross_left_start = geom.Polygon(cross_left_start)
# cross left side end
cross_left_end = np.array([[pitch_wyscout.right,
                             pitch_wyscout.bottom],
                            [pitch_wyscout.right - pitch_wyscout.penalty_area_length,
                             pitch_wyscout.bottom],
                            [pitch_wyscout.right - pitch_wyscout.penalty_area_length,
                             pitch_wyscout.top + pitch_wyscout.six_yard_from_side],
                            [pitch_wyscout.right,
                             pitch_wyscout.top + pitch_wyscout.six_yard_from_side]])
cross_left_end = geom.Polygon(cross_left_end)
# find intersection of passes and cross polygons
df_pass = df_wyscout_event[df_wyscout_event.pass_attempt].copy()
# starting locations
pass_start = geom.MultiPoint(df_pass[['x', 'y']].values)
cross_start_left_intersects = [point.intersects(cross_left_start) for point in pass_start]
cross_start_right_intersects = [point.intersects(cross_right_start) for point in pass_start]
# end locations
pass_end = geom.MultiPoint(df_pass[['end_x', 'end_y']].values)
cross_end_left_intersects = [point.intersects(cross_left_end) for point in pass_end]
cross_end_right_intersects = [point.intersects(cross_right_end) for point in pass_end]
# add cross marker to event data
mask_cross = ((np.array(cross_start_left_intersects) & np.array(cross_end_left_intersects)) | 
              (np.array(cross_start_right_intersects) & np.array(cross_end_right_intersects)))
cross_ids = df_pass[mask_cross].id
df_wyscout_event['cross'] = df_wyscout_event.id.isin(cross_ids)

Add on cut-back StatsBomb definition (StatsBomb definition: See Appendix 5 of the Docs for Events)

In [11]:
# right side start
cut_right_start = np.array([[pitch_wyscout.right, pitch_wyscout.bottom - pitch_wyscout.six_yard_from_side],
                            [pitch_wyscout.right, pitch_wyscout.bottom],
                            [pitch_wyscout.right - pitch_wyscout.six_yard_length, pitch_wyscout.bottom],
                            [pitch_wyscout.right - pitch_wyscout.six_yard_length,
                             pitch_wyscout.bottom - pitch_wyscout.six_yard_from_side]])
cut_right_start = geom.Polygon(cut_right_start)
# right side end
cut_right_end = np.array([[pitch_wyscout.right - pitch_wyscout.six_yard_length,
                          pitch_wyscout.bottom - pitch_wyscout.penalty_area_from_side],
                         [pitch_wyscout.right - pitch_wyscout.penalty_area_length,
                          pitch_wyscout.bottom - pitch_wyscout.penalty_area_from_side],
                         [pitch_wyscout.right - pitch_wyscout.penalty_area_length,
                          pitch_wyscout.top + pitch_wyscout.six_yard_from_side],
                         [pitch_wyscout.right - pitch_wyscout.six_yard_length,
                          pitch_wyscout.top + pitch_wyscout.six_yard_from_side]])
cut_right_end = geom.Polygon(cut_right_end)
# left side start
cut_left_start = np.array([[pitch_wyscout.right, pitch_wyscout.top + pitch_wyscout.six_yard_from_side],
                           [pitch_wyscout.right, pitch_wyscout.top],
                           [pitch_wyscout.right - pitch_wyscout.six_yard_length, pitch_wyscout.top],
                           [pitch_wyscout.right - pitch_wyscout.six_yard_length,
                            pitch_wyscout.top + pitch_wyscout.six_yard_from_side]])
cut_left_start = geom.Polygon(cut_left_start)
# left side end
cut_left_end = np.array([[pitch_wyscout.right - pitch_wyscout.six_yard_length,
                          pitch_wyscout.top + pitch_wyscout.penalty_area_from_side],
                         [pitch_wyscout.right - pitch_wyscout.penalty_area_length,
                          pitch_wyscout.top + pitch_wyscout.penalty_area_from_side],
                         [pitch_wyscout.right - pitch_wyscout.penalty_area_length,
                          pitch_wyscout.bottom - pitch_wyscout.six_yard_from_side],
                         [pitch_wyscout.right - pitch_wyscout.six_yard_length,
                          pitch_wyscout.bottom - pitch_wyscout.six_yard_from_side]])
cut_left_end = geom.Polygon(cut_left_end)
# find intersection of passes and cut back polygons
cut_start_left_intersects = [point.intersects(cut_left_start) for point in pass_start]
cut_start_right_intersects = [point.intersects(cut_right_start) for point in pass_start]
# end locations
cut_end_left_intersects = [point.intersects(cut_left_end) for point in pass_end]
cut_end_right_intersects = [point.intersects(cut_right_end) for point in pass_end]
# add cut back marker to event data
mask_cut = ((np.array(cut_start_left_intersects) & np.array(cut_end_left_intersects)) | 
            (np.array(cut_start_right_intersects) & np.array(cut_end_right_intersects)))
# not high and comes from a normal pass (not corner etc.)
cut_ids = df_pass[((mask_cut) & (df_pass.high == False) & (df_pass.eventName == 'Pass'))].id
df_wyscout_event['cut_back'] = df_wyscout_event.id.isin(cut_ids)

Add goal scored (excluding shootouts)

In [12]:
df_wyscout_event['goal_scored_excl_shootout'] = (((df_wyscout_event.goal) &
                                                  (df_wyscout_event['matchPeriod']!='P') &
                                                  (df_wyscout_event['shot'])) |
                                                 (df_wyscout_event.own_goal))

Check for missing Goals - there is one Kevin de Bruyne goal missing in the Wyscout event data

In [13]:
goals_per_game = pd.DataFrame(df_wyscout_event[df_wyscout_event['goal_scored_excl_shootout']]
                              .groupby('match_id')
                              .match_id.count())
goals_per_game.columns = ['Goals']
goals_per_game.reset_index(inplace=True)
goals_per_game = goals_per_game.merge(df_wyscout_match[['home_score', 'label', 'away_score','match_id', 'kick_off']], 
                                      how='right', on='match_id')
goals_per_game = goals_per_game.fillna(0)
# only one game missing a goal it's a Kevin de Bruyne goal and it's not in the event data
goals_per_game[goals_per_game.Goals != (goals_per_game.home_score + goals_per_game.away_score)]

,match_id,Goals,home_score,label,away_score,kick_off
1731,2499781,0.0,0.0,"Chelsea - Manchester City, 0 - 1",1.0,2017-09-30 18:30:00


#### Create a pass_height_name feature.

Assumptions:
- headed passes are high (roughly 60% are in the StatsBomb data)
- smart passes with the through ball tag are Ground/ low. It says in the Wyscout docs that through ball is added to smart pass if the pass is on the ground or it’s over the heads of the opposite players, but it’s on short distance – 5-10 meters.
- smart passes without through balls are high passes
- hand passes are ground/low (hopefully launch catches high passes)
- throw-in / goal kick are high
- free-kick, crosses and corners are low unless high=True

In [14]:
# assumption made here that head passes are high passes (in StatsBomb roughly 60% are)
df_wyscout_event.loc[df_wyscout_event.subEventName.isin(['High pass', 'Launch', 'Head pass']),
                     'pass_height_name'] = 'High Pass'
df_wyscout_event.loc[(df_wyscout_event.high == True) & (df_wyscout_event.eventName == 'Pass'),
                     'pass_height_name'] = 'High Pass'
# if smart pass and not a through ball assumed high
df_wyscout_event.loc[(df_wyscout_event.subEventName == 'Smart pass') & (df_wyscout_event.through == False),
                     'pass_height_name'] = 'High Pass'
df_wyscout_event.loc[df_wyscout_event.subEventName == 'Simple pass',
                     'pass_height_name'] = 'Ground/ Low Pass' 
df_wyscout_event.loc[(df_wyscout_event.subEventName.isin(['Corner', 'Free kick cross', 'Free Kick', 'Cross'])) &
                     (df_wyscout_event.high == False),
                     'pass_height_name'] = 'Ground/ Low Pass'
df_wyscout_event.loc[df_wyscout_event.subEventName.isin(['Throw in', 'Goal kick']),
                     'pass_height_name'] = 'High Pass'
df_wyscout_event.loc[(df_wyscout_event.subEventName.isin(['Corner', 'Free kick cross', 'Cross'])) &
                     (df_wyscout_event.high),
                     'pass_height_name'] = 'High Pass'
# assumption made here that smart through balls are ground/ low
df_wyscout_event.loc[(df_wyscout_event.subEventName == 'Smart pass') & (df_wyscout_event.through),
                     'pass_height_name'] = 'Ground/ Low Pass'
df_wyscout_event.loc[(df_wyscout_event.subEventName == 'Hand pass'),
                     'pass_height_name'] = 'Ground/ Low Pass'

Seperate out names in player dataset

In [15]:
df_wyscout_player['fullName'] = (df_wyscout_player.firstName + ' ' + df_wyscout_player.lastName).str.strip()
player_name_series = df_wyscout_player.fullName.str.split(' ')
df_wyscout_player['firstName'] = player_name_series.apply(lambda x: x[0] if isinstance(x, list) else None)
df_wyscout_player['middleName'] = player_name_series.apply(lambda x: ' '.join(x[1:-1]) if isinstance(x, list) else None)
df_wyscout_player['lastName'] = player_name_series.apply(lambda x: x[-1] if isinstance(x, list) else None)
df_wyscout_player['middleName'] = df_wyscout_player['middleName'].str.strip()
df_wyscout_player['Name'] = ((df_wyscout_player['firstName'] + ' ' + df_wyscout_player['middleName']).str.strip()
                             + ' ' + df_wyscout_player['lastName'])


Add on the player name/ foot

In [16]:
df_wyscout_player.foot.replace({'null': None, '': None}, inplace=True)
df_wyscout_event = df_wyscout_event.merge(df_wyscout_player[['player_id',
                                                             'firstName', 'middleName', 'lastName', 'Name', 'foot']],
                                          how='left', on='player_id')

Create a pass_technique name.

Assumptions:
- outswinging, different foot as the side of the pitch
- inswinging, same foot as the side of the pitch
- missing/ both foot = inswinging
- each player takes the kick with the foot in the player table.

This misses straight corner kicks and may not be 100% correct. 

In [17]:
mask_corner = df_wyscout_event.subEventName == 'Corner'
mask_right = df_wyscout_event.foot == 'right'
mask_left = df_wyscout_event.foot == 'left'
mask_right_side = df_wyscout_event.y >= pitch_wyscout.center_width
mask_left_side = df_wyscout_event.y < pitch_wyscout.center_width
mask_both = df_wyscout_event.foot == 'both'
mask_missing = df_wyscout_event.foot.isnull()
mask_inswing = mask_corner & ((mask_left & mask_left_side) | (mask_right & mask_right_side) | mask_both | mask_missing)
mask_outswing = mask_corner & ((mask_left & mask_right_side) | (mask_right & mask_left_side))
df_wyscout_event.loc[mask_inswing, 'pass_technique_name'] = 'Inswinging'
df_wyscout_event.loc[mask_outswing, 'pass_technique_name'] = 'Outswinging'
df_wyscout_event.loc[df_wyscout_event.through, 'pass_technique_name'] = 'Through Ball'

Fix a corner at the wrong end


In [18]:
df_wyscout_event.loc[(df_wyscout_event.y == 100) & (df_wyscout_event.x == 0) & 
                     (df_wyscout_event.subEventName == 'Corner'), 'x'] = 100

Length of the pass in yards

In [19]:
yards = (((df_wyscout_event.end_x - df_wyscout_event.x) / pitch_wyscout.right * 115) ** 2 +
         ((df_wyscout_event.end_y - df_wyscout_event.y) / pitch_wyscout.bottom * 74) ** 2) ** (0.5)
df_wyscout_event['pass_length'] = yards
df_wyscout_event.loc[(~df_wyscout_event.pass_attempt), 'pass_length'] = np.nan

Pass angle (StatsBomb definition)

In [20]:
x_dist = (df_wyscout_event.end_x - df_wyscout_event.x) / pitch_wyscout.right * 115
y_dist = (df_wyscout_event.y - df_wyscout_event.end_y) / pitch_wyscout.bottom * 74
# note negative angle so same as StatsBomb definition
# 0 pointing straight ahead, positve clockwise, negative anticlockwise
pass_radians = - np.arctan2(y_dist, x_dist)
df_wyscout_event['pass_angle'] = pass_radians
df_wyscout_event.loc[(~df_wyscout_event.pass_attempt), 'pass_angle'] = np.nan

Correct a few shots that appear to be on the wrong side of the pitch. I haven't checked, but these seem to be too far away to be real shots. Especially as some are goals

In [21]:
mask_correct_shot = (df_wyscout_event.x < 34) & (df_wyscout_event.shot)
df_wyscout_event.loc[mask_correct_shot, 'x'] = 100 - df_wyscout_event.loc[mask_correct_shot, 'x']

Fast attack, win ball in own third, shoot in last quarter in 7-25 seconds

In [22]:
mask_defence_win = ((df_wyscout_event.subEventName.isin(['Ground defending duel', 'Air duel', 'Save attempt'])) |
                    (df_wyscout_event.interception)) & (df_wyscout_event.x < 33.4)
df_wyscout_event.loc[mask_defence_win, 'defence_win'] = df_wyscout_event.loc[mask_defence_win, 'team_id']
df_wyscout_event.loc[mask_defence_win, 'defence_sec'] = df_wyscout_event.loc[mask_defence_win, 'eventSec']
group_match = df_wyscout_event.groupby(['match_id', 'matchPeriod'])
df_wyscout_event[['defence_win', 'defence_sec']] = group_match[['defence_win', 'defence_sec']].ffill()
mask_fast = (((df_wyscout_event.eventSec - df_wyscout_event.defence_sec) <= 25) &
             (df_wyscout_event.x > 75) &
             (df_wyscout_event.shot) & (df_wyscout_event.team_id == df_wyscout_event.defence_win))
df_wyscout_event['fast_break'] = mask_fast

Flag 10 seconds from a corner or freekick/ 20 seconds from a throw-in

In [23]:
for set_piece in ['Corner', 'Throw in', ['Free kick cross', 'Free kick shot']]:
    if isinstance(set_piece, list):
        mask = df_wyscout_event.subEventName.isin(set_piece)
        name = 'freekick'
    else:
        mask = df_wyscout_event.subEventName.isin([set_piece])
        name = set_piece.replace(' ', '_').lower()
    df_wyscout_event.loc[mask, f'{name}_sec'] = df_wyscout_event.loc[mask, 'eventSec']
    df_wyscout_event.loc[mask, f'{name}_team'] = df_wyscout_event.loc[mask, 'team_id']
    df_wyscout_event[f'{name}_sec'] = group_match[f'{name}_sec'].ffill()
    df_wyscout_event[f'{name}_team'] = group_match[f'{name}_team'].ffill()
    df_wyscout_event[f'{name}_sec'] = df_wyscout_event.eventSec - df_wyscout_event[f'{name}_sec']
df_wyscout_event.loc[df_wyscout_event.throw_in_sec > 20, 'throw_in_sec'] = np.nan
df_wyscout_event.loc[df_wyscout_event.freekick_sec > 10, 'freekick_sec'] = np.nan
df_wyscout_event.loc[df_wyscout_event.corner_sec > 10, 'corner_sec'] = np.nan
df_wyscout_event['play_type'] = df_wyscout_event[['throw_in_sec', 'freekick_sec', 'corner_sec']].idxmin(axis=1).str[:-4]
# if throw-in seperate defensive from offensive
mask_defensive = ((df_wyscout_event.play_type == 'throw_in') &
                  (df_wyscout_event['throw_in_team'] != df_wyscout_event.team_id))
df_wyscout_event.loc[mask_defensive, 'play_type'] = 'defensive_' + df_wyscout_event.loc[mask_defensive, 'play_type']

Add on previous info

In [24]:
# first filter out some events so the previous event is the correct assist type
mask_exclude = ((df_wyscout_event.eventName.isin(['Goalkeeper leaving line', 'Interruption'])) |
                (df_wyscout_event.subEventName == 'Acceleration'))
df_wyscout_event = df_wyscout_event[~mask_exclude].copy()
match_group = df_wyscout_event.groupby(['match_id', 'matchPeriod'])
for i in range(1, 4):
    df_wyscout_event[f'prev_id_{i}'] = match_group.id.shift(i)
    df_wyscout_event[f'prevEventName_{i}'] = match_group.eventName.shift(i)
    df_wyscout_event[f'prevSubEventName_{i}'] = match_group.subEventName.shift(i)
    df_wyscout_event[f'prev_player_id_{i}'] = match_group.player_id.shift(i)
    df_wyscout_event[f'prev_team_id_{i}'] = match_group.team_id.shift(i)
    df_wyscout_event[f'prev_pass_attempt_{i}'] = match_group.pass_attempt.shift(i)
    df_wyscout_event[f'prev_shot_{i}'] = match_group.shot.shift(i)

### Filter Non-penalty/ non-corner shots

In [25]:
mask_shot1 = (((df_wyscout_event.eventName=='Shot') | (df_wyscout_event.subEventName=='Free kick shot')) & 
              (df_wyscout_event['matchPeriod']!='P'))
mask_shot2 = (df_wyscout_event.subEventName=='Corner') & (df_wyscout_event.goal==True)
mask_corner_goal = (df_wyscout_event.subEventName=='Corner') & (df_wyscout_event.goal==True)
mask_shot = (mask_shot1 | mask_shot2) & (~mask_corner_goal)
df_wyscout_shots = df_wyscout_event[mask_shot].copy()
print('Number of shots:', len(df_wyscout_shots))
print('Number of goals:', df_wyscout_shots.goal.sum())

Number of shots: 42867
Number of goals: 4384


Add body part name

In [26]:
df_wyscout_shots.loc[df_wyscout_shots.left_foot, 'body_part_name'] = 'Left Foot'
df_wyscout_shots.loc[df_wyscout_shots.right_foot, 'body_part_name'] = 'Right Foot'
df_wyscout_shots.loc[df_wyscout_shots.other_body_part, 'body_part_name'] = 'Other'
df_wyscout_shots.drop(['left_foot', 'right_foot', 'other_body_part'], axis=1, inplace=True)

Strongest foot column

In [27]:
mask_strong_foot = (((df_wyscout_shots.foot.isin(['right', 'both'])) & (df_wyscout_shots.body_part_name == 'Right Foot')) |
                    (((df_wyscout_shots.foot.isin(['left', 'both'])) & (df_wyscout_shots.body_part_name == 'Left Foot'))))
df_wyscout_shots['strong_foot'] = mask_strong_foot

Add shot type name

In [28]:
# note there are three goals that come direct from corner kicks - they are tagged 'Free Kick' originally though
# this was renamed earlier to Set Piece
df_wyscout_shots['shot_type_name']  = df_wyscout_shots.eventName.replace({'Shot': 'Open Play',
                                                                          'Set Piece': 'Direct Set Piece'})

Assist type, and get event id if pass

In [29]:
# direct from set pieces
df_wyscout_shots.loc[df_wyscout_shots.shot_type_name == 'Direct Set Piece', 'assist_type'] = 'direct'
# rebound/ clearance from the previous event
df_wyscout_shots.loc[(df_wyscout_shots.assist_type.isnull()) &
                     (df_wyscout_shots.prevEventName_1.isin(['Save attempt', 'Shot', 'Set Piece'])), 'assist_type'] = 'rebound'
df_wyscout_shots.loc[(df_wyscout_shots.assist_type.isnull()) &
                     (df_wyscout_shots.prevSubEventName_1 == 'Clearance'), 'assist_type'] = 'clearance'
# pass from the previous event
mask_pass1 = ((df_wyscout_shots.prev_pass_attempt_1 == True) &
              (df_wyscout_shots.team_id == df_wyscout_shots.prev_team_id_1) &
              (df_wyscout_shots.assist_type.isnull()))
df_wyscout_shots.loc[mask_pass1, 'assist_type'] = 'pass'
df_wyscout_shots.loc[mask_pass1, 'pass_id'] = df_wyscout_shots.loc[mask_pass1, 'prev_id_1']
# pass from the third previous event if and there are two duels in-between
mask_duel = ((df_wyscout_shots.prevEventName_1 == 'Duel') &
             (df_wyscout_shots.prevEventName_2 == 'Duel') &
             (df_wyscout_shots.assist_type.isnull()))             
mask_pass2 = (mask_duel & (df_wyscout_shots.prev_pass_attempt_3) &
              (df_wyscout_shots.team_id == df_wyscout_shots.prev_team_id_3))
df_wyscout_shots.loc[mask_pass2, 'assist_type'] = 'pass'
df_wyscout_shots.loc[mask_pass2, 'pass_id'] = df_wyscout_shots.loc[mask_pass2, 'prev_id_3'] 
# rebound/clearance if the third previous event involved a shot or save and there are two duels in-between
df_wyscout_shots.loc[(df_wyscout_shots.assist_type.isnull()) & mask_duel &
                     ((df_wyscout_shots.prev_shot_3) | (df_wyscout_shots.prevEventName_3 == 'Save attempt')),
                     'assist_type'] = 'rebound'
df_wyscout_shots.loc[(df_wyscout_shots.assist_type.isnull()) & mask_duel &
                     (df_wyscout_shots.prevSubEventName_3 =='Clearance'),
                     'assist_type'] = 'clearance'
# if still null and the second event involves a shot or save set to a rebound
df_wyscout_shots.loc[(df_wyscout_shots.assist_type.isnull()) & 
                     ((df_wyscout_shots.prevEventName_2 == 'Save attempt') | (df_wyscout_shots.prev_shot_2)),
                     'assist_type'] = 'rebound'
# if still null and the second event involves a clearance set to a clearance
df_wyscout_shots.loc[df_wyscout_shots.assist_type.isnull() & (df_wyscout_shots.prevSubEventName_2 == 'Clearance'),
                     'assist_type'] = 'clearance'
# if still null and the third event involves a pass set to a pass
mask_pass3 = ((df_wyscout_shots.assist_type.isnull()) & (df_wyscout_shots.prev_pass_attempt_3) & 
              (df_wyscout_shots.prev_team_id_3 == df_wyscout_shots.team_id))
df_wyscout_shots.loc[mask_pass3, 'assist_type'] = 'pass'
df_wyscout_shots.loc[mask_pass3, 'pass_id'] = df_wyscout_shots.loc[mask_pass3, 'prev_id_3']
# if still null set to recovery
df_wyscout_shots.loc[df_wyscout_shots.assist_type.isnull(), 'assist_type'] = 'recovery'

Keep subset of columns

In [30]:
df_wyscout_shots = df_wyscout_shots[['match_id', 'id', 'goal', 'team_id', 'team_name', 'player_id',
                                     'firstName', 'middleName', 'lastName', 'Name', 
                                     'shot_type_name', 'play_type',
                                     'x', 'y', 'counter_attack', 'fast_break',
                                     'strong_foot', 'body_part_name', 'assist_type', 'pass_id']].copy()                               

Add on pass information

In [31]:
df_pass = df_wyscout_event.loc[df_wyscout_event.pass_attempt, ['id', 'end_y', 'end_x',
                                                               'pass_switch', 'cross', 'cut_back', 'pass_height_name',
                                                               'pass_technique_name', 'pass_length', 'pass_angle']].copy()
df_pass.rename({'id': 'pass_id', 'end_x': 'pass_end_x', 'end_y': 'pass_end_y',
                'cross': 'pass_cross', 'cut_back': 'pass_cut_back'}, axis=1, inplace=True)
df_wyscout_shots = df_wyscout_shots.merge(df_pass, how='left', on='pass_id')

Calculate distance of carry/ dribble in yards (not calculated as too often zero)

In [32]:
#yards = (((df_wyscout_shots.pass_end_x - df_wyscout_shots.x) / pitch_wyscout.right * 115) ** 2 +
#         ((df_wyscout_shots.pass_end_y - df_wyscout_shots.y) / pitch_wyscout.bottom * 74) ** 2) ** (0.5)
#df_wyscout_shots['carry_length'] = yards

Side of field

In [33]:
df_wyscout_shots.loc[df_wyscout_shots.y < 45, 'side'] = 'left'
df_wyscout_shots.loc[(df_wyscout_shots.y >= 45) & (df_wyscout_shots.y <= 55), 'side'] = 'center'
df_wyscout_shots.loc[df_wyscout_shots.y > 55, 'side'] = 'right'

Convert coordinates to standard pitch size (105m * 68m)

In [34]:
x_cols = ['x', 'pass_end_x']
y_cols = ['y', 'pass_end_y']
df_wyscout_shots[x_cols] = (df_wyscout_shots[x_cols]) / float(pitch_wyscout.right) * pitch_statsperform.right
df_wyscout_shots[y_cols] = ((float(pitch_wyscout.bottom) - df_wyscout_shots[y_cols]) /
                            float(pitch_wyscout.bottom)  * pitch_statsperform.top)

Droping the end locations of the assist pass as they too often are the same as the shot location (compared with StatsBomb where they differ more often)

In [35]:
df_wyscout_shots.drop(['pass_end_y', 'pass_end_x'], axis=1, inplace=True)

Angles/ distance to goals

In [36]:
left_post, right_post = pitch_statsperform.goal_right
goal_width = abs(right_post - left_post)[1]
dx = abs(pitch_statsperform.right - df_wyscout_shots.x)
dy = abs(pitch_statsperform.center_width - df_wyscout_shots.y)
df_wyscout_shots['visible_angle'] = np.arctan2(goal_width * dx , (dx**2 + dy**2 - (goal_width / 2.) ** 2))
df_wyscout_shots['middle_angle'] = np.arctan2(dy, dx)
df_wyscout_shots['distance_to_goal'] = round((dy**2 + dx**2)**0.5, 1)

Interaction between angle and distance

In [37]:
df_wyscout_shots['distance_mid_angle'] = df_wyscout_shots.distance_to_goal * df_wyscout_shots.middle_angle
df_wyscout_shots['distance_visible_angle'] = df_wyscout_shots.distance_to_goal * df_wyscout_shots.visible_angle

Log distance

In [38]:
df_wyscout_shots['log_distance_to_goal'] = np.log(df_wyscout_shots.distance_to_goal)

Amend shot type to take into account the play_type (set piece column) made earlier

In [39]:
mask_amend = (df_wyscout_shots.shot_type_name != 'Direct Set Piece') & (df_wyscout_shots.play_type.notnull())
df_wyscout_shots.loc[mask_amend, 'shot_type_name'] = df_wyscout_shots.loc[mask_amend, 'play_type']
df_wyscout_shots['shot_type_name'] = df_wyscout_shots.shot_type_name.str.lower().str.replace(' ', '_')
df_wyscout_shots.drop('play_type', axis=1, inplace=True)

Add Men

In [40]:
df_wyscout_shots['competition_gender'] = 'male'

Save dataset

In [41]:
df_wyscout_shots.drop(['pass_id'], axis=1, inplace=True)
df_wyscout_shots.reset_index(drop=True, inplace=True)
df_wyscout_shots.to_parquet(os.path.join(WYSCOUT, 'shots.parquet'))

Info on dataset

In [42]:
df_wyscout_shots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42867 entries, 0 to 42866
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   match_id                42867 non-null  int64  
 1   id                      42867 non-null  int64  
 2   goal                    42867 non-null  bool   
 3   team_id                 42867 non-null  int64  
 4   team_name               42867 non-null  object 
 5   player_id               42864 non-null  float64
 6   firstName               42864 non-null  object 
 7   middleName              42864 non-null  object 
 8   lastName                42864 non-null  object 
 9   Name                    42864 non-null  object 
 10  shot_type_name          42867 non-null  object 
 11  x                       42867 non-null  float64
 12  y                       42867 non-null  float64
 13  counter_attack          42867 non-null  bool   
 14  fast_break              42867 non-null